In [0]:
%run ../Includes/School-Setup

In [0]:
%sql
select * from students

In [0]:
%sql
describe students

In [0]:
%sql
select student_id,profile:first_name,profile:address:country from students

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW parsed_students AS
 SELECT student_id, from_json(profile, schema_of_json('{"first_name":"Sarah",
 "last_name":"Lundi", "gender":"Female", "address":{"street":"8 Greenbank Road",
 "city":"Ottawa", "country":"Canada"}}')) AS profile_struct
 FROM students;

SELECT * FROM parsed_students;


In [0]:
%sql
describe parsed_students

In [0]:
%sql
select student_id,profile_struct.first_name,profile_struct.address.country from parsed_students;

In [0]:
%sql
create or replace temp view students_final as
select student_id,profile_struct.*,profile_struct.address.* from parsed_students;
    
select * from students_final;

In [0]:
%sql
select enroll_id,student_id,courses from enrollments;

In [0]:
%sql
select enroll_id, student_id, explode(courses) as course from enrollments;
    
select * from students_final;

In [0]:
%sql
select * from enrollments

In [0]:
%sql
describe enrollments

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW parsed_enrollments AS
 select enroll_id, student_id, explode(courses) 
  as course from enrollments;



In [0]:
%sql
select * from parsed_enrollments;

In [0]:
%sql
select 
enroll_id,student_id,
course.course_id,course.discount_percent, 
course.subtotal
from parsed_enrollments;

In [0]:
%sql
select * from enrollments;

In [0]:
%sql
select student_id,
collect_set(courses.course_id) as before_flatten,
array_distinct(flatten(collect_set(courses.course_id))) as after_flatten
from enrollments
group by student_id
order by student_id;


In [0]:
%sql
select student_id,
collect_set(enroll_id) as enrollments_set,
collect_set(courses.course_id) as courses_set
from enrollments
group by student_id
order by student_id;

## Working Wiht Joins

In [0]:
%sql
select * from enrollments;

In [0]:
%sql
select * from courses;

In [0]:
%sql
create or replace view enrollments_enriched as
select * from (
  select *, explode(courses) as course_json
  from enrollments) e
  inner join courses c
  on e.course_json.course_id = c.course_id


In [0]:
%sql
select * from enrollments_enriched;

In [0]:
%sql
create or replace temp view enrollments_updates
as select * from parquet.`${dataset.school}/enrollments-new`

In [0]:
%sql
select * from enrollments
union all
select * from enrollments_updates

In [0]:
%sql
select * from enrollments
intersect
select * from enrollments_updates;